### Imports

In [1]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os, sys
from pathlib import Path

os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
sys.path.append('/home/k64835/Master-Thesis-SITS')

scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))

scripts_path = Path("../Evaluation/").resolve()
sys.path.append(str(scripts_path))

In [8]:
import pickle
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from scripts.data_visualiser import *
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.model_helper import *
from model_scripts.pretrained_temporal_feature_extraction import *
from model_scripts.dataset_creation import *
from model_scripts.train_model_ae import *
from model_scripts.model_visualiser import *
from evaluation_scripts.patch_evaluation_helper import get_clustering_accuracy
from model_scripts.clustering import *
from model_scripts.subpatch_extraction import *
from evaluation_scripts.result_visualiser import *
from evaluation_scripts.evaluation_helper import *
from Pipeline.temporal_preprocessing_pipeline import *
import numpy as np
import config
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import skimage.measure
import torch
import torch.nn as nn
import torch.optim as optim

## Dataset prep: B10

### Loading the pre-processed data

Data: Extracted and Pre-processed Patches (each patch containing a sugarbeet field)

Dimensions: (N, T, C, H, W) = (N, 7, 10, 64, 64)

In [ ]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, images_visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'b10')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, images_visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'b10')
patch_tensor_train.shape, patch_tensor_eval.shape

### Create Sub-Patches

In [ ]:
train_subpatches, train_subpatch_coords = non_overlapping_sliding_window(patch_tensor_train, field_numbers_train, patch_size=config.subpatch_size)
eval_subpatches, eval_subpatch_coords = non_overlapping_sliding_window(patch_tensor_eval, field_numbers_eval, patch_size=config.subpatch_size)
train_subpatches.shape, eval_subpatches.shape

Get field numbers and co-ordinates as string

In [ ]:
train_coord_dataloader = get_string_fielddata(train_subpatch_coords)
eval_coord_dataloader = get_string_fielddata(eval_subpatch_coords)
train_coord_dataloader[0]

### Split Unlabeled data into 'train' and 'test' and create  Data Loaders

In [ ]:
train_subpatches_dl, test_subpatches, train_field_numbers, test_field_numbers = train_test_split(
    train_subpatches, train_coord_dataloader, test_size=1-config.ae_train_test_ratio, random_state=42
)

dataloader_train = create_data_loader(train_subpatches_dl, train_field_numbers, batch_size=config.ae_batch_size, shuffle=True)
dataloader_test = create_data_loader(test_subpatches, test_field_numbers, batch_size=config.ae_batch_size, shuffle=False)
dataloader_eval = create_data_loader(eval_subpatches, eval_coord_dataloader, batch_size=config.ae_batch_size, shuffle=False)

## Pre-trained Model 1: Resnet 3D trained on Video Data

### Extract Features

In [ ]:
device = 'cuda'

In [ ]:
timesformer_extractor = ResNet3DFeatureExtractor()
tf_features_train, train_coord_dl = extract_features(timesformer_extractor, dataloader_train, device)
print("ResNet3D Extracted Features Shape:", tf_features_train.shape)  

In [ ]:
tf_features_test, test_coord_dl = extract_features(timesformer_extractor, dataloader_test, device)
print("ResNet3D Extracted Features Shape:", tf_features_test.shape)  

In [ ]:
tf_features_eval, eval_coord_dl = extract_features(timesformer_extractor, dataloader_eval, device)
print("ResNet3D Extracted Features Shape:", tf_features_eval.shape)  

### K-means + Evaluation

In [ ]:
kmeans = train_kmeans_patches(tf_features_train.cpu(), n_clusters=2, random_state=12)

train_patch_predictions = kmeans.predict(tf_features_train.reshape(tf_features_train.size(0), -1).numpy().astype(np.float32))
test_patch_predictions = kmeans.predict(tf_features_test.reshape(tf_features_test.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(tf_features_eval.reshape(tf_features_eval.size(0), -1).numpy().astype(np.float32))

In [ ]:
# Assign field labels
threshold = 0.5
train_field_labels = assign_field_labels_ae(train_coord_dl, train_patch_predictions, threshold)
test_field_labels = assign_field_labels_ae(test_coord_dl, test_patch_predictions, threshold)
eval_field_labels = assign_field_labels_ae(eval_coord_dl, eval_patch_predictions, threshold)

In [ ]:
acc, precision, recall, f1_score, f2_score = evaluate_clustering_metrics(eval_field_labels, config.labels_path)
print("Accuracy:",acc)
print("Precision:",precision)
print("Recall:",recall)
print("F1-score:",f1_score)
print("F2-score:", f2_score)

## Pre-trained Model 2: Earthformer

### Use patch level images

In [ ]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'rgb')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'rgb')
patch_tensor_train.shape, patch_tensor_eval.shape

In [ ]:
resized_tensor_train = resize_images_transfer_learning(patch_tensor_train, (224, 224))
resized_tensor_eval = resize_images_transfer_learning(patch_tensor_eval, (224,224))
resized_tensor_train.shape, resized_tensor_eval.shape

### Create Patch-level Data Loaders

In [ ]:
train_patches_dl, test_patches, train_field_numbers, test_field_numbers = train_test_split(
    resized_tensor_train, field_numbers_train, test_size=1-0.75, random_state=10
)

dataloader_train = create_data_loader(train_patches_dl, train_field_numbers, batch_size=config.ae_batch_size, shuffle=True)
dataloader_test = create_data_loader(test_patches, test_field_numbers, batch_size=config.ae_batch_size, shuffle=False)
dataloader_eval = create_data_loader(resized_tensor_eval, field_numbers_eval, batch_size=config.ae_batch_size, shuffle=False)

### Extract Features

In [ ]:
device = 'cuda'
timesformer_extractor = EarthformerFeatureExtractor()
tf_features_train, train_coord_dl = extract_features(timesformer_extractor, dataloader_train, device)
print("ResNet3D Extracted Features Shape:", tf_features_train.shape)  

In [ ]:
tf_features_test, test_coord_dl = extract_features(timesformer_extractor, dataloader_test, device)
print("ResNet3D Extracted Features Shape:", tf_features_test.shape)  

In [ ]:
tf_features_eval, eval_coord_dl = extract_features(timesformer_extractor, dataloader_eval, device)
print("ResNet3D Extracted Features Shape:", tf_features_eval.shape)  

### K-means + Evaluation

In [ ]:
kmeans = train_kmeans_patches(tf_features_train.cpu(), n_clusters=2, random_state=13)

train_patch_predictions = kmeans.predict(tf_features_train.reshape(tf_features_train.size(0), -1).numpy().astype(np.float32))
test_patch_predictions = kmeans.predict(tf_features_test.reshape(tf_features_test.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(tf_features_eval.reshape(tf_features_eval.size(0), -1).numpy().astype(np.float32))

In [ ]:
acc, precision_per_class, recall_per_class, f1_per_class, f2_score = get_clustering_accuracy(field_numbers_eval, eval_patch_predictions, config.labels_path)
print("Accuracy:",acc)
print("Precision:",precision_per_class)
print("Recall:",recall_per_class)
print("F1-score:",f1_per_class)
print("F2-score",f2_score)

## Pre-trained Model 3: Timesformer

### Use patch level images

In [9]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'rgb')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'rgb')
patch_tensor_train.shape, patch_tensor_eval.shape

(torch.Size([2425, 7, 3, 64, 64]), torch.Size([48, 7, 3, 64, 64]))

In [10]:
resized_tensor_train = resize_images_transfer_learning(patch_tensor_train, (224, 224))
resized_tensor_eval = resize_images_transfer_learning(patch_tensor_eval, (224,224))
resized_tensor_train.shape, resized_tensor_eval.shape

(torch.Size([2425, 7, 3, 224, 224]), torch.Size([48, 7, 3, 224, 224]))

### Create Patch-level Data Loaders

In [11]:
train_patches_dl, test_patches, train_field_numbers, test_field_numbers = train_test_split(
    resized_tensor_train, field_numbers_train, test_size=1-0.75, random_state=10
)

dataloader_train = create_data_loader(train_patches_dl, train_field_numbers, batch_size=config.ae_batch_size, shuffle=True)
dataloader_test = create_data_loader(test_patches, test_field_numbers, batch_size=config.ae_batch_size, shuffle=False)
dataloader_eval = create_data_loader(resized_tensor_eval, field_numbers_eval, batch_size=config.ae_batch_size, shuffle=False)

/home/k64835/Master-Thesis-SITS/Modeling/model_scripts/dataset_creation.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32).permute(0, 2, 1, 3, 4)   # (N, T, C, H, W) -> (N, C, T, H, W)


### Extract Features

In [21]:
device = 'cuda'
timesformer_extractor = PretrainedTimeSformerFeatureExtractor1()
tf_features_train, train_coord_dl = extract_features(timesformer_extractor, dataloader_train, device)
print("ResNet3D Extracted Features Shape:", tf_features_train.shape)  

RuntimeError: shape '[64, 12, 14, 8, 768]' is invalid for input of size 67436544

In [ ]:
import timm
timm.list_models()

In [ ]:
tf_features_test, test_coord_dl = extract_features(timesformer_extractor, dataloader_test, device)
print("ResNet3D Extracted Features Shape:", tf_features_test.shape)  

In [ ]:
tf_features_eval, eval_coord_dl = extract_features(timesformer_extractor, dataloader_eval, device)
print("ResNet3D Extracted Features Shape:", tf_features_eval.shape)  

### K-means + Evaluation

In [ ]:
kmeans = train_kmeans_patches(tf_features_train.cpu(), n_clusters=2, random_state=12)

train_patch_predictions = kmeans.predict(tf_features_train.reshape(tf_features_train.size(0), -1).numpy().astype(np.float32))
test_patch_predictions = kmeans.predict(tf_features_test.reshape(tf_features_test.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(tf_features_eval.reshape(tf_features_eval.size(0), -1).numpy().astype(np.float32))

In [ ]:
# Assign field labels
threshold = 0.5
train_field_labels = assign_field_labels_ae(train_coord_dl, train_patch_predictions, threshold)
test_field_labels = assign_field_labels_ae(test_coord_dl, test_patch_predictions, threshold)
eval_field_labels = assign_field_labels_ae(eval_coord_dl, eval_patch_predictions, threshold)

In [ ]:
acc, precision, recall, f1_score, f2_score = evaluate_clustering_metrics(eval_field_labels, config.labels_path)
print("Accuracy:",acc)
print("Precision:",precision)
print("Recall:",recall)
print("F1-score:",f1_score)
print("F2-score:", f2_score)

## Pre-trained Models: ConvLSTM -- remove

In [ ]:
convlstm_extractor = ConvLSTMFeatureExtractor()
convlstm_features_train, train_coord_dl = extract_features(convlstm_extractor, dataloader_train, device)
print("ConvLSTM Extracted Features Shape:", convlstm_features_train.shape)


In [ ]:
convlstm_features_test, test_coord_dl = extract_features(convlstm_extractor, dataloader_test, device)
print("ResNet3D Extracted Features Shape:", convlstm_features_test.shape)  

In [ ]:
convlstm_features_eval, eval_coord_dl = extract_features(convlstm_extractor, dataloader_eval, device)
print("ResNet3D Extracted Features Shape:", convlstm_features_eval.shape)  